In [2]:
import os
import re
import gzip
import json
import matplotlib.pyplot as plt
# name of channel to be scanned
channel_prefix = "https://www.youtube.com/"
#channel = "@StronnyCuttles"
channel = "@ShibiCottonbum"
#channel = "@ImmyBisou"
#channel = "@IceySnowpaws"
#channel = "@MercyModiste"
#channel = "@AzuraDulait"

In [3]:
try:   os.mkdir(channel)
except:   pass
try:   os.mkdir(f"{channel}/downloaded")
except:   pass
try:   os.mkdir(f"{channel}/processed")
except:   pass
try:   os.mkdir(f"{channel}/transcripts")
except:   pass
# file containing keywords
keywords_file = "keywords.txt"

# file containing thematic searches
thematic_file = "thematic.txt"

# scan channel for videos
# live videos
os.system(f"yt-dlp --get-id --skip-download {channel_prefix+channel} > {channel}/videos.txt")
videos = []
with open(f"{channel}/videos.txt", "r") as f:
    for line in f:
        if line.startswith("Error"):
            continue
        videos.append(line.strip())

# look for filenames in {channel}/downloaded for already downloaded videos
# the file name contains the video id in the following format:
# 【Supermarket Simulator】Definitely not drunk on the job 🐐🍼【VAllure】 [i3rrDSjokF4].webm
# where [i3rrDSjokF4] is the video id
downloaded = []
for file in os.listdir(f"{channel}/downloaded"):
    if file.endswith(".webm"):
        m = re.search(r"\[(.*?)\]", file)
        if m:
            downloaded.append(m.group(1))
            
# do the same for transcripts
transcripts = []
for file in os.listdir(f"{channel}/transcripts"):
    if file.endswith(".srt"):
        m = re.search(r"\[([^\[\]]+)\](?!.*\[)", file)
        if m:
            transcripts.append(m.group(1))


# compare list of videos to videos already downloaded
to_download = list(set(videos) - set(downloaded) - set(transcripts))
for download in to_download:
    print(f"{download}")

WmFvJ9NbjVs
C2Up8kSa4lM
i7GN2f10VIQ
8v3Jj4ItmzE
iOCjBHb4t50
GGhc8LeQqSw
df5OIRbwWJU
PF_dHkE1_rI
jDT-SCtswq0
DafZTKcNKCg
j7bTZIMNkVw
Wh9CmkbYLMY
8uuXS7l1LnE
ts68uC4-cI4
nFWmrZnP_SI
yE3sJZUa4XM
KzDUmFGef3c
Bth6F5wbaP8
7RPhHyYIEo0
xA0TXX3roDg
yK8s8ZOrku0
OEA6tIjKplA
MvyhWT9AApM
17St8KhMArg
gTbusIo3cnk
6NZZuLTc5gs
CO562tfp7kU
5vXJTNCGiiA
5N86tMvqi4c
NokRLOmFGa4


In [4]:
newly_downloaded = []
for video in to_download:
    print(f"Downloading {video}")
    #output = os.system(f"yt-dlp -f bestaudio -P {channel}/downloaded {channel_prefix}watch?v={video}")
    output = os.system(f"yt-dlp --write-subs --sub-lang live_chat -f bestaudio -P {channel}/downloaded {channel_prefix}watch?v={video}")
    if output!=0:
        print(f"Error downloading {video}")

Error downloading iOCjBHb4t50


In [5]:
#collect file names in {channel}/downloaded
downloaded_files = os.listdir(f"{channel}/downloaded")
for file in downloaded_files:
    if file.endswith(".webm") or file.endswith(".m4a"):
        #result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --output_dir --language English --model large-v3 \"{channel}/transcripts/\"")
        result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --language English --output_dir \"{channel}/transcripts/\"")
        if result==0:
            #move file to {channel}/processed
            try:
                os.rename(f"{channel}/downloaded/{file}", f"{channel}/processed/{file}")
            except:
                print(f"Error moving {file} to {channel}/processed")

: 

In [3]:
from collections import defaultdict
from datetime import timedelta


def create_histogram(data, window_size_seconds):
    """
    Create a histogram of the chat messages based on the given window size.
    """
    # Calculate start and end times
    start_time = min(data)
    end_time = max(data)
    
    # Initialize the bins for the histogram
    bins = defaultdict(int)
    current_window_start = start_time

    while current_window_start <= end_time:
        next_window_start = current_window_start + timedelta(seconds=window_size_seconds)
        # Count messages in the current window
        for timestamp in data:
            if current_window_start <= timestamp < next_window_start:
                bins[current_window_start] += 1
        current_window_start = next_window_start

    # Plot the histogram
    plt.bar(bins.keys(), bins.values(), width=timedelta(seconds=window_size_seconds), align='edge')
    plt.xlabel('Time')
    plt.ylabel('Number of Chat Messages')
    plt.title(f'Chat Messages Histogram (Window Size: {window_size_seconds} seconds)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [5]:
# move and transform chat scripts. we want to extract user, timestamp and message
# chat scripts are in the format:
#{"clickTrackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB", "replayChatItemAction": {"actions": [{"clickTrackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB", "addChatItemAction": {"item": {"liveChatTextMessageRenderer": {"message": {"runs": [{"emoji": {"emojiId": "🩷", "shortcuts": [":pink_heart:"], "searchTerms": ["pink", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/1fa77/72.png"}], "accessibility": {"accessibilityData": {"label": "🩷"}}}}}, {"emoji": {"emojiId": "🤍", "shortcuts": [":white_heart:"], "searchTerms": ["white", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/1f90d/72.png"}], "accessibility": {"accessibilityData": {"label": "🤍"}}}}}, {"emoji": {"emojiId": "🤎", "shortcuts": [":brown_heart:"], "searchTerms": ["brown", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/1f90e/72.png"}], "accessibility": {"accessibilityData": {"label": "🤎"}}}}}, {"emoji": {"emojiId": "❤", "shortcuts": [":red_heart:", ":heart:", "<3"], "searchTerms": ["red", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/2764/72.png"}], "accessibility": {"accessibilityData": {"label": "❤"}}}}}]}, "authorName": {"simpleText": "TappedPotential🐰👠"}, "authorPhoto": {"thumbnails": [{"url": "https://yt4.ggpht.com/mAj0LNN_j5Bv3kr0iTP6bNsE-BIwkoUXCqIOR1zahwcF7Q46xBzVp3JrCuBkwGhRRmO8_0YOlA=s32-c-k-c0x00ffffff-no-rj", "width": 32, "height": 32}, {"url": "https://yt4.ggpht.com/mAj0LNN_j5Bv3kr0iTP6bNsE-BIwkoUXCqIOR1zahwcF7Q46xBzVp3JrCuBkwGhRRmO8_0YOlA=s64-c-k-c0x00ffffff-no-rj", "width": 64, "height": 64}]}, "contextMenuEndpoint": {"clickTrackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB", "commandMetadata": {"webCommandMetadata": {"ignoreNavigation": true}}, "liveChatItemContextMenuEndpoint": {"params": "Q2g0S0hBb2FRMHRwV1RCMlRHWnlOR2RFUmxSclN6Rm5RV1JJVW1OSlFWRWFLU29uQ2hoVlEzWlplbmRGYTBOUk1qZG5VR0ZxTlRSM1FuTldUMmNTQzNSek5qaDFRelF0WTBrMElBRW9BVElhQ2hoVlF6bDBjVjlpYlVSbU9YYzVNblpFYm1ReWIwdzJNbWM0QWtnQlVBRSUzRA=="}}, "id": "ChwKGkNLaVkwdkxmcjRnREZUa0sxZ0FkSFJjSUFR", "timestampUsec": "1725674881850547", "authorBadges": [{"liveChatAuthorBadgeRenderer": {"customThumbnail": {"thumbnails": [{"url": "https://yt3.ggpht.com/LROENIDC2LQf2oAC1WcICEsPbihMHhp5APeKA9jks871KM4K8lrSQN_uoK85gmE3c8qmjFc96Q=s16-c-k", "width": 16, "height": 16}, {"url": "https://yt3.ggpht.com/LROENIDC2LQf2oAC1WcICEsPbihMHhp5APeKA9jks871KM4K8lrSQN_uoK85gmE3c8qmjFc96Q=s32-c-k", "width": 32, "height": 32}]}, "tooltip": "Member (2 months)", "accessibility": {"accessibilityData": {"label": "Member (2 months)"}}}}], "authorExternalChannelId": "UC9tq_bmDf9w92vDnd2oL62g", "contextMenuAccessibility": {"accessibilityData": {"label": "Chat actions"}}, "timestampText": {"simpleText": "-8:48"}, "trackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB"}}, "clientId": "CKiY0vLfr4gDFTkK1gAdHRcIAQ"}}], "videoOffsetTimeMsec": "0"}}
#show debug if:
# has more than 1 action
# has more than 1 run
# doesn't have user
# videoOffsetTimeMsec is lower than 0 or bigger than 7200000 (max 12 hours)
# doesn't have message
files = os.listdir(f"{channel}/downloaded")
chat_files = [i for i in files if i.endswith("file_chat.json")]
print(chat_files)
for file_path in chat_files:
    all_timestamps = []
    anomalies = []
    with open(file_path, 'r', encoding='utf-8') as file:
        try:
            # Load the JSON content
            for line in file:
                json_data = json.loads(line.strip())
                for item in json_data.get('replayChatItemAction', {}).get('actions', []):
                    # Extract the timestamp
                    if 'addChatItemAction' in item:
                        chat_item = item['addChatItemAction']['item']
                        if 'liveChatTextMessageRenderer' in chat_item:
                            timestamp = chat_item['liveChatTextMessageRenderer']['timestampUsec']
                        elif 'liveChatViewerEngagementMessageRenderer' in chat_item:
                            timestamp = chat_item['liveChatViewerEngagementMessageRenderer']['timestampUsec']
                        else:
                            continue

                        # Convert to datetime object
                        parsed_timestamp = parse_timestamp(timestamp)
                        
                        # Filter for timestamps that are on the 7th or 8th
                        if parsed_timestamp.day in [7, 8]:
                            all_timestamps.append(parsed_timestamp)
                        else:
                            anomalies.append(line.strip())  # Keep track of anomalous lines
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error processing file {file_path}: {e}")

    # Create a histogram if any timestamps were collected
    if all_timestamps:
        create_histogram(all_timestamps, window_size_seconds)

    # Display anomalous lines if any
    if anomalies:
        print(f"\nAnomalous lines detected ({len(anomalies)} lines):")
        for anomaly in anomalies:
            print(anomaly)

[]


In [5]:
#remove .tsv, .txt, .vtt files, (keeping .srt and .json), and compress json files to .json.gz in {channel}/transcripts
transcript_files = os.listdir(f"{channel}/transcripts")
for file in transcript_files:
    if file.endswith(".tsv") or file.endswith(".txt") or file.endswith(".vtt"):
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
    if file.endswith(".json"):
        with open(f"{channel}/transcripts/{file}", "rb") as f:
            with gzip.open(f"{channel}/transcripts/{file}.gz", "wb") as f_out:
                f_out.writelines(f)
        #remove original json file
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
        

In [2]:
videos = ["rifLvjFgjc4"]
for video in videos:
    print(f"Downloading {video}")
    output = os.system(f"yt-dlp --write-subs --sub-lang live_chat --skip-download {channel_prefix}watch?v={video}")
    if output!=0:
        print(f"Error downloading {video}")

#yt-dlp --write-subs --sub-lang live_chat --write-info-json --skip-download <VIDEO_URL>
